In [1]:
from ultralytics import YOLO
from pathlib import Path

import pandas as pd
from tqdm import tqdm

/home/simba9/miniconda3/envs/ultralytics/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
file_image_lists = list(Path("./test_dataset").rglob("*.jpg"))

In [3]:
model = YOLO("/home/simba9/pet/shov/runs/detect/0.8648825873387556 data_old_run_1080_full/weights/best.pt")

In [4]:
results = model(file_image_lists[0]) 


image 1/1 /home/simba9/pet/shov/test_dataset/12.jpg: 640x1088 2 adjs, 61.7ms
Speed: 7.3ms preprocess, 61.7ms inference, 273.4ms postprocess per image at shape (1, 3, 640, 1088)


In [5]:
results[0].names

{0: 'adj', 1: 'int', 2: 'geo', 3: 'pro', 4: 'non'}

In [6]:
#dir(results[0])

In [7]:
df = pd.DataFrame({
    "filename": [],
    "class_id": [],
    "rel_x": [],
    "rel_y": [],
    "width": [],
    "height": []
})

In [8]:
for image_path in tqdm(file_image_lists):
    results = model(image_path, verbose=False)
    for box, cls  in zip(results[0].boxes, results[0].boxes.cls):
        x,y,w,h = box.xywhn.cpu().numpy().tolist()[0]
        cls = int(cls.cpu().item())
        df = pd.concat([pd.DataFrame([[image_path.name, cls, x,y,w,h]], columns=df.columns), df], ignore_index=True)
df.class_id = df.class_id.astype(int)

100%|██████████| 363/363 [00:04<00:00, 86.07it/s]


In [9]:
df.to_csv("sumbission.csv", index=False, sep=";")